# Compare balance areas to model projections

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr
from tqdm.auto import tqdm
from scipy.stats import median_abs_deviation as MAD, mode
import sys
import seaborn as sns
# Suppress future warning from pandas
import warnings
warnings.filterwarnings("ignore")
import matplotlib

In [ ]:
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping-application/'
sys.path.append(os.path.join(base_path, 'functions'))
import model_analyze_utils as f

scm_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/'

# load glacier area projection file names
fns = sorted(glob.glob(os.path.join(scm_path, 'Rounce_et_al_2023', 'area_projections', '*.nc')))
# separate into projections and MAD files
proj_fns = sorted([x for x in fns if 'mad' not in x])
mad_fns = [x for x in fns if 'mad' in x]

# load glacier outlines
aois_fn = os.path.join(scm_path, 'compiled_data', 'all_aois_climate_cluster.shp')
aois = gpd.read_file(aois_fn)
aois[['O1Region', 'O2Region']] = aois[['O1Region', 'O2Region']].astype(int)

# load balance areas
balance_areas_fn = os.path.join(scm_path, 'results', 'balanced_areas.csv')
balance_areas = pd.read_csv(balance_areas_fn)
balance_areas

In [ ]:
# -----Compare balance areas to the lowest GHG scenarios
# Subset file names to lowest GHG scenarios
rcp26_fns = [x for x in proj_fns if 'rcp26' in x]
rcp26_mad_fns = [x for x in mad_fns if 'rcp26' in x]
ssp119_fns = [x for x in proj_fns if 'ssp119' in x]
ssp119_mad_fns = [x for x in mad_fns if 'ssp119' in x]
# Load and concatenate files for R01 and R02
rcp26 = xr.open_dataset(rcp26_fns[0])
rcp26 = xr.concat([rcp26, xr.open_dataset(rcp26_fns[1])], dim='glacier')
rcp26_mad = xr.open_dataset(rcp26_mad_fns[0])
rcp26_mad = xr.concat([rcp26_mad, xr.open_dataset(rcp26_mad_fns[1])], dim='glacier')
ssp119 = xr.open_dataset(ssp119_fns[0])
ssp119 = xr.concat([ssp119, xr.open_dataset(ssp119_fns[1])], dim='glacier')
ssp119_mad = xr.open_dataset(ssp119_mad_fns[0])
ssp119_mad = xr.concat([ssp119_mad, xr.open_dataset(ssp119_mad_fns[1])], dim='glacier')

In [ ]:
ssp119_fns

In [ ]:
# Plot balance area vs. modeled 2100 area for all sites
areas_df = pd.DataFrame()
for id in tqdm(balance_areas['RGIId'].drop_duplicates().values):
    # subset balance areas and projections
    balance_area = balance_areas.loc[balance_areas['RGIId']==id]
    I_rcp26_glacier = np.ravel(np.argwhere(rcp26.RGIId.data==id))
    rcp26_glacier = rcp26.isel(glacier=I_rcp26_glacier)
    I_ssp119_glacier = np.ravel(np.argwhere(ssp119.RGIId.data==id))
    ssp119_glacier = ssp119.isel(glacier=I_ssp119_glacier)
    # grab subregion and cluster
    subregion = aois.loc[aois['RGIId']==id, 'Subregion'].values[0]
    cluster = aois.loc[aois['RGIId']==id, 'clustName'].values[0]
    # concatenate in dataframe
    df = pd.DataFrame({'RGIId': [id],
                       'Subregion': [subregion], 
                       'Cluster': [cluster],
                       'Balance area [km2]': [balance_area['Balance area [km2]'].values[0]],
                       '2100 mean modeled area, RCP26 [km2]': rcp26_glacier.sel(year=2100).glac_area_annual.mean(dim='model').data / 1e6,
                       '2100 mean modeled area, SSP119 [km2]': ssp119_glacier.sel(year=2100).glac_area_annual.mean(dim='model').data / 1e6
                      })
    areas_df = pd.concat([areas_df, df])
    
areas_df.reset_index(drop=True, inplace=True)

# Plot with colors distinguishing subregions and clusters
for column in ['Subregion', 'Cluster']:
    fig, ax = plt.subplots(1, 2, figsize=(12,6))
    for i, proj in enumerate(['RCP26', 'SSP119']):
        sns.scatterplot(data=areas_df, x='Balance area [km2]', y='2100 mean modeled area, ' + proj + ' [km2]', 
                        hue=column, legend=True, ax=ax[i])
        line_max = np.round(np.max(np.ravel(areas_df[['Balance area [km2]', f'2100 mean modeled area, {proj} [km2]']].values)))
        ax[i].plot(np.arange(0, line_max), np.arange(0, line_max), '-k', linewidth=1)
        ax[i].set_xlabel('Balance area [km$^2$]')
        ax[i].set_ylabel(f'{proj} 2100 mean modeled area [km$^2$]')
        ax[i].grid()
    plt.show()


In [ ]:
# Plot NORMALIZED balance area vs. modeled 2100 area for all sites,
# i.e. divide the previous by the current RGI area
areas_df = pd.DataFrame()
for id in tqdm(balance_areas['RGIId'].drop_duplicates().values):
    # subset balance areas and projections
    area = balance_areas.loc[balance_areas['RGIId']==id, 'Area [km2]'].values[0]
    balance_area = balance_areas.loc[balance_areas['RGIId']==id, 'Balance area [km2]'].values[0]
    I_rcp26_glacier = np.ravel(np.argwhere(rcp26.RGIId.data==id))
    rcp26_glacier = rcp26.isel(glacier=I_rcp26_glacier)
    I_ssp119_glacier = np.ravel(np.argwhere(ssp119.RGIId.data==id))
    ssp119_glacier = ssp119.isel(glacier=I_ssp119_glacier)
    # grab subregion and cluster
    subregion = aois.loc[aois['RGIId']==id, 'Subregion'].values[0]
    cluster = aois.loc[aois['RGIId']==id, 'clustName'].values[0]
    # concatenate in dataframe
    df = pd.DataFrame({'RGIId': [id],
                       'Subregion': [subregion],
                       'Cluster': [cluster],
                       'Balance area, normalized': [balance_area / area],
                       '2100 mean modeled area, RCP26, normalized': rcp26_glacier.sel(year=2100).glac_area_annual.mean(dim='model').data / 1e6 / area,
                       '2100 mean modeled area, SSP119, normalized': ssp119_glacier.sel(year=2100).glac_area_annual.mean(dim='model').data / 1e6 / area
                      })
    areas_df = pd.concat([areas_df, df])
    
areas_df.reset_index(drop=True, inplace=True)

# Plot with colors distinguishing subregions and clusters
for column in ['Subregion', 'Cluster']:
    fig, ax = plt.subplots(1, 2, figsize=(12,6))
    for i, proj in enumerate(['RCP26', 'SSP119']):
        sns.scatterplot(data=areas_df, x='Balance area, normalized', y='2100 mean modeled area, ' + proj + ', normalized', 
                        hue=column, legend=True, ax=ax[i])
        if i==0:
            leg = ax[i].get_legend()
            leg.remove()
            ax[i].set_title('RCP 2.6: ~1.5-2$^\circ$C warming')
        else:
            ax[i].set_title('SSP 1-1.9: ~1-1.8$^\circ$C warming')
        line_max = np.round(np.max(np.ravel(areas_df[['Balance area, normalized', f'2100 mean modeled area, {proj}, normalized']].values)))
        ax[i].plot(np.arange(0, line_max), np.arange(0, line_max), '-k', linewidth=1)
        ax[i].set_xlabel('Balance area, normalized')
        ax[i].set_ylabel(f'{proj} 2100 mean modeled area, normalized')
        ax[i].grid()
    plt.show()
    # Save figure
    fig_fn = os.path.join(scm_path, 'snow-cover-mapping-application', 'figures', f'balance_areas_comparison_normalized_{column}.png')
    fig.savefig(fig_fn, dpi=250)
    print('Figure saved to file:', fig_fn)
           

In [ ]:
# Plot distribution of balance areas
fig, ax = plt.subplots(2, 1, figsize=(10,12))
sns.histplot(data=areas_df, x='Balance area, normalized', hue='Subregion', 
             bins=np.arange(0, 2, step=0.2), multiple='stack', ax=ax[0])
sns.histplot(data=areas_df, x='Balance area, normalized', hue='Cluster', 
             bins=np.arange(0, 2, step=0.2), multiple='stack', ax=ax[1])
plt.show()

In [ ]:
# Print stats
print('Balance areas: ')
print('\tMedian:', np.nanmedian(areas_df['Balance area, normalized']))
print('\tMAD:', MAD(areas_df['Balance area, normalized']))
print('\tMean:',  np.nanmean(areas_df['Balance area, normalized']))
print('\tstd.:', np.nanstd(areas_df['Balance area, normalized']))
print('\n+ balance areas, N =', len(areas_df.loc[areas_df['Balance area, normalized'] > 1]))
print('- balance areas, N =', len(areas_df.loc[areas_df['Balance area, normalized'] < 1]))